# Initialize
This section is a developer aid; TODO remove later!
*Align with [storyboard Version 48](https://rubinobs.atlassian.net/wiki/pages/viewpage.action?pageId=132612364&pageVersion=48)*

If you are using this under Time Square and *get any errors* that you think come from this notebook, please tell Steve Pothier (steve.pothier@noirlab.edu).  Include the URL you used to get the error. 

In [1]:
# Parameters.
# Times Square replaces this cell with the user's parameters.
# So, don't put anything else here!

# day_obs values: TODAY, YESTERDAY, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
#!day_obs = '2024-09-25' # Value to use for local testing (Summit)
day_obs = "YESTERDAY"  # TODO Change to 'YESTERDAY' to test with default before push

# Total number of days of data to display (ending on day_obs)
number_of_days = "1"  # TODO Change to '1' to test with default before push

In [2]:
import datetime as dt
import os
from collections import defaultdict
from pprint import pformat, pp
from urllib.parse import urlencode, urlparse
import pandas as pd
from IPython.display import HTML

# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get("EXTERNAL_INSTANCE_URL"):
    dev_msg = (
        'Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch. \n'
        'TODO: Make the need for this go away by getting Logging_and_Reporting installed in RSP.'
    )
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype > /dev/null 2>&1
else:
    dev_msg = 'Imported lsst.ts.logging_and_reporting from local packages.'
from lsst.ts.logging_and_reporting.all_sources import AllSources
from lsst.ts.logging_and_reporting.all_reports import AllReports
import lsst.ts.logging_and_reporting.utils as ut
from lsst.ts.logging_and_reporting.reports import md, mdlist, mdpathlink
from lsst.ts.logging_and_reporting.reports import html_draft, html_beta

In [3]:
# Set default env to "usdf" and try before PUSH to repo.
# The default provided here is for local testing. Under Times Square it is ignored.
server = os.environ.get(
    "EXTERNAL_INSTANCE_URL", 
    ut.Server.usdf) # TODO try with "usdf" before push (else "summit")

In [4]:
# Normalize Parameters (both explicit Times Squares params, in implicit ones)
limit = 5000  # YAGNI for Auto get more if this isn't enough to get all requested DAYS
response_timeout = 3.05  # seconds, how long to wait for connection
read_timeout = 20  # seconds
timeout = (float(response_timeout), float(read_timeout))

date = ut.get_datetime_from_dayobs_str(day_obs)
# date:  is EXCLUSIVE (upto, but not including)
days = int(number_of_days)

# Thus: [min_day_obs,max_day_obs)
# Format: string, YYYY-MM-DD
min_date = date - dt.timedelta(days=days - 1)
max_date = date + dt.timedelta(days=1)
min_day_obs = min_date.strftime("%Y-%m-%d")  # Inclusive
max_day_obs = max_date.strftime("%Y-%m-%d")  # prep for Exclusive

# Experimental Features config
exp_tally = "true"  # default: true
exp_parsing = "true"  # default: true
experimental = dict()
experimental["exposure_type_tally"] = exp_tally == "true"
experimental["error_parsing"] = exp_parsing == "true"

In [5]:
# Read records from (almost) all sources
allsrc = AllSources(
    server_url=server,
    min_dayobs=min_day_obs,
    max_dayobs=max_day_obs,
    verbose=True,                # TODO change to False before push
    limit=limit,
    exclude_instruments = []      # TODO change to empty list before push
)
allrep = AllReports(allsrc=allsrc)

DEBUG validate_response endpoint_url='https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?limit=2'
DBG get_records endpoint='https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports'
DBG get_records qstr: is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241107&max_day_obs=20241108
DEBUG validate_response endpoint_url='https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241107&max_day_obs=20241108'
DBG get_records len(page)=2 len(recs)=0
DBG get_records-2 len(page)=2 len(recs)=2
DEBUG validate_response endpoint_url='https://usdf-rsp-dev.slac.stanford.edu/exposurelog/messages?limit=2'
DEBUG validate_response endpoint_url='https://usdf-rsp-dev.slac.stanford.edu/exposurelog/instruments'
DBG get_exposures endpoint='https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures'
DBG get_exposures qstr: registry=1&instrument=LSSTComCamSim&order_by=-timespan_end&offset

In [6]:
instrum_str = ", ".join(list(allsrc.exp_src.instruments.keys()))
md(f"# Showing data for {min_date.date()} to {max_date.date()} for {instrum_str}")

# Showing data for 2024-11-07 to 2024-11-08 for LSSTComCamSim, LATISS, LSSTComCam, LSSTCam

# Table of Contents
* [Night Report](#Night-Report-BETA)
    - AuxTel
    - Simonyi
* [Almanac](#Almanac-BETA)
* [Summary plots of whole night](#Summary-plots-of-whole-night-DRAFT)
* [Time Accounting](#Time-Accounting-DRAFT)
* [Jira Tickets](#Jira-Tickets-BETA)
    - AuxTel
    - Simonyi
* [BLOCKS Observed](#BLOCKS-Observed-DRAFT)
    - AuxTel
    - Simonyi
* [Data Log](#Data-Log-DRAFT)
* [Narrative Log](#Narrative-Log-BETA)
* [Developer Only Section](#Developer-Only-Section-REMOVE)

## Night Report <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [7]:
# Night Report
# Display time log
allrep.nig_rep.time_log_as_markdown()

### Telescope: AuxTel

```

```

Telescope Status: 

*Authors: *

### Telescope: Simonyi

```
The day started with powering up the rotator and the hexapods. The worming up was scheduled for the late afternoon. After checking with Jacques and Brian, we agreed to leave the temperature set point to 11C. Information from Freddy: There is an auxiliary pump close to OSS . Hoses are connected but system is DISABLED (filter case open) and is not related AT ALL with the operation of the telescope. However, the open filter was dripping oil to a unusual rate (we had this information from the beginning of the day, to be aware of). At 2:05pm (CLT) we had a discuttion with Brian and Te-Wei about worming up the hexapods and we decided to do it a bit earlier. While M2Hexapods are warming up, Marina, Gonzalo and I went up to Level 8 for the TMA and Dome Handover. The handover went smoothly. Right after the handover we got confirmation from Petr that we are good to go on HardPoint Breakaway tests (BLOCK-T145), that is started right away. The hardpoint breakaway test was performed on the EUI.
After the hardpoint test we ran the bunmp test and during this time the elevation drives failed and David Jimenez recovered the TMA. After bump test we opened the mirror covers to perform a laser tracker test. In the meantime Gabrielle ran the M2 bump test which triggered some alarms (?) which were resolved quickly. Then, we attempted to perform the laser tracker alignment, but the script ran too quickly. The mirror covers got stuck when we tried to close them, but this issue was resolved by running the command again.

We experienced a minor earthquake near the start of the late shift ~9:15pm.  After the earthquake, MTM1M3, MTMount, and MTPtg went to fault and we recovered them.  After that, we could not solve the MTDome telemetry issues.  After three hours of troubleshooting, we opted to close the dome and perform closed-dome tests, including the TMA Azimuth and Elevation Brake Distance - 2% Settings tests.  We had an issue with the EIB which we resolved by rebooting the encoder hardware in the EUI.  We then proceeded to run the Block T-227 Dynamic Test at 34 deg, but ran into issues when passing the configuration in the move_p2p.py script.  Our solution was to manually add each p2p position to the script queue and run.  We didn't experience any issues with this approach.  After this completed, we transitioned to park the TMA and end the night.  We lowered the mirror.  The only issue we had when ending the night was when we sent MTM1M3 to disabled by CSC.  We cleared the interlock and put MTM1M3 in disabled.
```

Telescope Status: The final state is:
HVAC set point = 11 deg C 
We kept on the FCU with 300 RPM 
MTM1M3 DISABLED - lowered and disabled
MTM2 ENABLED in closed loop
MTAOS STANDBY
MTDome DISABLED
MTDomeTrajectory STANDBY
MTMount DISABLED
MTHexapods in STANDBY w/ cabinets and PXIs powered off
MTPtg ENABLED
MTRotator at 0 deg, in STANDBY w/ cabinets and PXIs powered off


TMA is at the zenith position, Az = -32
OSS Power ON 
Power supply OFF 
Dome position 328 deg
Mirror covers are closed 
Locking pins in lock position 

*Authors: Bruno Quint, Marina Pavlovic, Gonzalo Aravena, Kate Napier, Carlos Morales*

## Almanac <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [8]:
# Display various almanac values (for moon, sun)
#!rep.AlmanacReport().day_obs_report(min_day_obs)
allrep.alm_rep.day_obs_report()

**Almanac for the observing night starting 2024-11-07**

,(times in UTC),
Moon Rise,2024-11-07 14:26:17.894,
Moon Set,2024-11-07 04:07:59.999,
Moon Illumination,29%,(% illuminated)
Astronomical Twilight (morning),2024-11-07 08:14:31.199,(-18 degrees)
Astronomical Twilight (evening),2024-11-07 00:38:30.190,(-18 degrees)
Solar Midnight,2024-11-07 04:26:30.748,
Nautical Twilight (morning),2024-11-07 08:46:31.869,(-12 degrees)
Nautical Twilight (evening),2024-11-07 00:06:29.446,(-12 degrees)
Civil Twilight (morning),2024-11-07 09:17:12.957,(-6 degrees)
Civil Twilight (evening),2024-11-06 23:35:48.246,(-6 degrees)


## Summary plots of whole night <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

(content not yet defined in storyboard)

In [9]:
# Plot Observation (Exposure) gaps
# rollup,detail = allsrc.get_observation_gaps()
# allrep.plot_observation_gap_detail(detail)

## Time Accounting <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
TODO:
- Time-loss for Fault and Weather
- SlewTime from TMAEvent (or similar)

SlewTime (and probably others) from EFD are only EXPECTED (topic="lsst.sal.Scheduler.logevent_target"), not ACTUAL.
To get ACTUAL time, we need to use something like TMAEvent to combine multiple real events into a synthetic event.  See DM-46102.


In [10]:
# Exposure Tally
tally = await allsrc.night_tally_observation_gaps()
if tally:
    display(pd.DataFrame(tally))

,LSSTCam,LSSTComCam
Total Night,7:36:01,7:36:01
Total Exposure,3:03:38,0:01:32
Slew time(1),0:00:00,0:00:00
Readout time(2),0:31:48,0:00:34
Time loss to fault,NA,NA
Time loss to weather,NA,NA
Idle time,4:32:22,7:34:28
Number of exposures,830,15
Mean readout time,0.000639,0.000639
Number of slews(1),0,0


- (1) *Expected* slew time as per Schedular
- (2) There is no practical way to get detector read-out time.  A value of 2.3 seconds was inferred from experiments.

## Jira Tickets <font style="background-color:green; color:white; font-size:20px">BETA</font> 
(Observing Operations - OBS)
(TODO: better tickets using Jira API)

In [11]:
if allsrc.urls:
    for url in allsrc.urls:
        md(f"- {mdpathlink(url)}")

- [OBS-321](https://rubinobs.atlassian.net/browse/OBS-321)

- [obs-321](https://rubinobs.atlassian.net/browse/obs-321)

In [12]:
# Display Jira BLOCKS
front = "https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/"
tickets = allsrc.nig_src.nightly_tickets()
if tickets:
    mdstr = "#### Nightly Jira BLOCKs"
    for day, url_list in tickets.items():
        mdstr += f"\n- {day}"
        for ticket_url in url_list:
            mdstr += f'\n    - [{ticket_url.replace(front,"")}]({ticket_url})'
    md(mdstr)
else:
    endpoint_url = allsrc.nig_src.status["reports"]["endpoint_url"]
    md(f"No tickets found using: [API Data]({endpoint_url}) in `confluence_url`.")

No tickets found using: [API Data](https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241107&max_day_obs=20241108) in `confluence_url`.

## BLOCKS Observed <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
(TODO: BLOCKS observed) 

## Data Log <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

NOTE: *modified all_sources.py:uniform_field_counts()*
NOTE: Each tally count is now a link to a detailed table of exposures.

In [13]:
# Exposure Report
if experimental.get("exposure_type_tally"): 
    #  md("### Experimental Exposure field Tally ENABLED")
    md("Tally exposures with specific values of selected fields")
else:
    md("### Experimental Exposure field Tally DISASBLED")
    
for instrum in allsrc.exp_src.exposures.keys():
    md(f'**{instrum}: {len(allsrc.exp_src.exposures[instrum])} total exposures**')
    if 0 == len(allsrc.exp_src.exposures[instrum]):
        continue

    # Facet counts
    if experimental.get("exposure_type_tally"): 
        counts,cc,f = allsrc.uniform_exposure_field_counts(instrum)
        for fname, tally in []: # counts.items():
            md(f"{fname}")
            df = pd.DataFrame.from_dict(tally, orient="index").T
            wdf = ut.wrap_dataframe_columns(df)
            display(HTML(wdf.to_html(render_links=True, escape=False)))

Tally exposures with specific values of selected fields

**LSSTComCamSim: 0 total exposures**

**LATISS: 0 total exposures**

**LSSTComCam: 15 total exposures**

DEBUG uniform_exposure eflag_fields=['good', 'questionable', 'junk', 'unknown'] use_fields=['observation_reason', 'observation_type', 'science_program']


**LSSTCam: 830 total exposures**

DEBUG uniform_exposure eflag_fields=['good', 'questionable', 'junk', 'unknown'] use_fields=['observation_reason', 'observation_type', 'science_program']


In [14]:
pd.DataFrame(cc['observation_type'])

,flat,bias,dark,unknown
0,266.0,194.0,369.0,1
1,NaN,NaN,NaN,266
2,NaN,NaN,NaN,194
3,NaN,NaN,NaN,369
4,NaN,NaN,NaN,1


Note: *Added counts of Exposures with messages*

In [15]:
for instrum, recs in allsrc.exp_src.messages.items():
    md(f'**{instrum} {len(recs)} Exposures with messages**')
    if len(recs) > 0:
        df = allsrc.tally_exposure_flags(instrum)
        display(df.style.hide(axis="index"))

## Narrative Log <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [16]:
# Narrative Report
# Time Log
allrep.nar_rep.time_log_as_markdown()

- **2024-11-07 08:48:49.461019** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Shut down rotator</pre>


- **2024-11-07 08:44:35.085235** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Turning off hexapods</pre>


- **2024-11-07 08:41:25.795068** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Taking ComCam daytime calibrations, BLOCK-T220, while telescope is slewing to park position.</pre>


- **2024-11-07 08:34:46.875485** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Going to telescope parking position. Elevation 90, azimuth -32.</pre>


- **2024-11-07 08:29:50.458408** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MTMount']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>h3. Error/Failure explanation:
Internal Alarm in Elevation OMT 08:26:05.572 07/11/24 code: 123 system:123
A timeout was ocurred when:<PARSED_NL>Power off softmotion axis. In state: "Disabling Axis"
h3. What were you doing when the failure occurred?:
While control over from EUI, elevation axis was fault. 
h3. Detailed description of the steps followed that caused the error/failure:
Reset alarm.</pre>


...

- **2024-11-07 17:03:53.871075** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Jacques, Brian, Marina, Gonzalo and I had a quick chat and we all agreed to leave the set point today at 11ºC.</pre>


- **2024-11-07 16:07:41.295262** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>running BLOCK-T246 Taking bias images</pre>


- **2024-11-07 16:03:18.586752** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Rotator and Hexapods startup went well with no issues</pre>


- **2024-11-07 15:43:51.013520** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Gonzalo and Marina are now in the control room. Both are powering up the hexapods and the rotator. We might leave the warm-ups for later following Te-Wei's suggestion ([here|https://rubin-obs.slack.com/archives/C07QJMQ01FU/p1730994153557769]). 


Meanwhile, I am addressing the temperatures in preparation for tonight.</pre>


- **2024-11-07 14:46:42.064697** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>There was a fault on M1M3 just now and I recovered. 
It was associated with work inside the cell where Felipe D. closed the DAA438 valve while checking for leaks.</pre>


-----------
------------

# Developer Only Section <font style="background-color:red; olor:black; font-size:20px">REMOVE</font>
Contains stuff only expected to be useful to developers.

This may also contain sections that have moved out of the user section because they are no longer defined in the Storyboard.

## Overview 


In [17]:
# Display overview of Report context
try:
    import lsst.ts.logging_and_reporting.version

    lrversion = lsst.ts.logging_and_reporting.version.__version__
except:
    lrversion = "LIVE"

try:
    from lsst.summit.utils import ConsDbClient

    have_consdb = True
except:
    have_consdb = False
md(
    f"""## Project-Wide Night(s) Report 
- Run on logs and databases from **{server}/**
- Report **{days} observing night(s)** with the last reported night starting on **{date}**.
- min_dayobs={allsrc.min_dayobs!r}, max_dayobs={allsrc.max_dayobs!r}
- Using ***Prototype* Logging and Reporting** Version: **{lrversion}**
- {have_consdb = }
- {dev_msg}
""")

## Project-Wide Night(s) Report 
- Run on logs and databases from **https://usdf-rsp-dev.slac.stanford.edu/**
- Report **1 observing night(s)** with the last reported night starting on **2024-11-07 12:00:00**.
- min_dayobs='2024-11-07', max_dayobs='2024-11-08'
- Using ***Prototype* Logging and Reporting** Version: **0.1.dev145+gf2a5765.d20241014**
- have_consdb = True
- Imported lsst.ts.logging_and_reporting from local packages.


## Data Status
This may be useful for diagnosing reports that give unexpected results.

In [18]:
df = pd.DataFrame(allsrc.get_data_status())
table = df.to_html(
    index=False, 
    render_links=True,
    escape=False)
display(HTML(table))

Endpoint,Records,URL
nightreport/reports,2,https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241107&max_day_obs=20241108
exposurelog/instruments,3,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/instruments
exposurelog/exposures.LSSTComCamSim,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTComCamSim&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241107&max_day_obs=20241108
exposurelog/exposures.LATISS,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LATISS&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241107&max_day_obs=20241108
exposurelog/exposures.LSSTComCam,15,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTComCam&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241107&max_day_obs=20241108
exposurelog/exposures.LSSTCam,830,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTCam&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241107&max_day_obs=20241108
exposurelog/messages,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/messages?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=2500&min_day_obs=20241107&max_day_obs=20241108
narrativelog/messages,149,https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_added&offset=0&limit=1000&min_date_added=2024-11-07T00%3A00%3A00&max_date_added=2024-11-08T00%3A00%3A00
efd/targets,0,NA


## This report uses the following data sources
- NightReport
- Exposurelog
- Narrativelog
- EFD
- ConsDB
- (DDV)
- (Almanac from Astroplan)

## DDV 

In [19]:
DDV = (
    f"{server}/rubintv-dev/ddv/index.html"
    if "summit" in server
    else f"{server}/rubintv/ddv/index.html"
)
md(f"Access DDV part of RubinTV: {DDV}")

Access DDV part of RubinTV: https://usdf-rsp-dev.slac.stanford.edu/rubintv/ddv/index.html

## Where was this run?

The environments that have everything needed to run this page are:

    https://summit-lsp.lsst.codes
    https://usdf-rsp-dev.slac.stanford.edu

However, Times Square does not run on the Summit. It does run on USDF-dev.


## Section overviews moved here

In [20]:
# Night Report Overview
allrep.nig_rep.overview()
# Exposure Report Overview
allrep.exp_rep.overview()
# Narrative Report Overview
allrep.nar_rep.overview()

### Overview for Service: `nightreport` [2]

None


- Endpoint: https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241107&max_day_obs=20241108

None
- Got 2 records.  


### Overview for Service: `exposurelog` [0]

...

None
- Got 0 records.  


### Overview for Service: `narrativelog` [149]

None


- Endpoint: https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_added&offset=0&limit=1000&min_date_added=2024-11-07T00%3A00%3A00&max_date_added=2024-11-08T00%3A00%3A00

None
- Got 149 records.  


In [21]:
# Conditionally display our current ability to connect to all needed endpoints.
if False and not os.environ.get("EXTERNAL_INSTANCE_URL"):
    md("## Dashboard")
    md("(This is not done when running under Times Square.)")
    %run ./dashboard.ipynb

### According to Storybaord, this section should be removed ...
... and replaced with a simple table tally of Good, Questionable, and  Junk exposures per instrument.
| Symbol | Meaning |
|:---|:---|
| <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>| Good |
| <font style="background-color:yellow; color:black; font-size:20px">&nbsp;?&nbsp;</font> | Questionable |
| <font style="background-color:red; color:black; font-size:20px">&nbsp;R&nbsp;</font> | Junk |


In [22]:
# Time Log
allrep.exp_rep.time_log_as_markdown()

## Finale

In [23]:
print(f"Finished {str(dt.datetime.now())}")

Finished 2024-11-08 16:10:09.194956
